WEB SCRAPPING TRANSFERMARKT

In [59]:
import lxml
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
import csv

In [15]:
# URL de la página de Mbappé en Transfermarkt
url = 'https://www.transfermarkt.es/kylian-mbappe/profil/spieler/342229'

# Hacemos la solicitud con un user-agent válido para no ser bloqueados
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')


In [56]:
# Extraer el nombre completo
try:
    headline_wrapper = soup.find('h1', class_='data-header__headline-wrapper')
    numero = headline_wrapper.find('span', class_='data-header__shirt-number').text.strip()
    nombre = headline_wrapper.text.replace(numero, "").strip()
except AttributeError:
    nombre = "Nombre no encontrado"

# Extraer la información del fichaje
try:
    club_info = soup.find('div', class_='data-header__club-info')
    
    # Nombre del equipo
    equipo = club_info.find('span', class_='data-header__club').find('a').text.strip()
    
    # Liga
    liga = club_info.find('span', class_='data-header__league').text.strip()
    
    # Fecha de fichaje
    fecha_fichaje = club_info.find_all('span', class_='data-header__label')[1].find('span', class_='data-header__content').text.strip()
    
    # Contrato hasta
    contrato_hasta = club_info.find_all('span', class_='data-header__label')[2].find('span', class_='data-header__content').text.strip()

except AttributeError:
    equipo = liga = fecha_fichaje = contrato_hasta = "Información no encontrada"

# Extraer la fecha de nacimiento y edad
try:
    birth_date_span = soup.find('span', {'itemprop': 'birthDate'})
    if birth_date_span:
        fecha_nacimiento = birth_date_span.get_text(strip=True)
    else:
        fecha_nacimiento = "Fecha de nacimiento no encontrada"
except AttributeError:
    fecha_nacimiento = "Fecha de nacimiento no encontrada"

# Extraer el lugar de nacimiento
try:
    birth_place_span = soup.find('span', {'itemprop': 'birthPlace'})
    if birth_place_span:
        lugar_nacimiento = birth_place_span.get_text(strip=True)
    else:
        lugar_nacimiento = "Lugar de nacimiento no encontrado"
except AttributeError:
    lugar_nacimiento = "Lugar de nacimiento no encontrado"

# Extraer la nacionalidad
try:
    nationality_span = soup.find('span', {'itemprop': 'nationality'})
    if nationality_span:
        nacionalidad = nationality_span.get_text(strip=True).split('\n')[-1].strip()
    else:
        nacionalidad = "Nacionalidad no encontrada"
except AttributeError:
    nacionalidad = "Nacionalidad no encontrada"

# Extraer la altura
try:
    height_span = soup.find('span', {'itemprop': 'height'})
    if height_span:
        altura = height_span.get_text(strip=True)
    else:
        altura = "Altura no encontrada"
except AttributeError:
    altura = "Altura no encontrada"

# Extraer la posición
try:
    # Encontrar todos los <ul> con la clase 'data-header__items'
    uls = soup.find_all('ul', class_='data-header__items')
    if len(uls) > 1:
        # Dentro del segundo <ul>, buscar el <li> que contiene 'Posición:'
        position_li = next((li for li in uls[1].find_all('li', class_='data-header__label') 
                            if 'Posición:' in li.get_text()), None)
        if position_li:
            position_span = position_li.find('span', class_='data-header__content')
            #posicion = get_text_or_default(position_span)
        else:
            posicion = "Posición no encontrada"
    else:
        posicion = "Posición no encontrada"
except AttributeError:
    posicion = "Posición no encontrada"

# Extraer la selección
try:
    # Encontrar el <li> que contiene 'Selección:'
    selection_li = next((li for li in soup.find_all('li', class_='data-header__label') 
                         if 'Selección:' in li.get_text()), None)
    if selection_li:
        selection_span = selection_li.find('span', class_='data-header__content')
        if selection_span:
            # Extraer y limpiar el texto de la selección
            selection = selection_span.get_text(strip=True)
            # Limpiar el texto de posibles elementos adicionales (imágenes y enlaces)
            selection = ' '.join(selection.split())
        else:
            selection = "Selección no encontrada"
    else:
        selection = "Selección no encontrada"
except AttributeError:
    selection = "Selección no encontrada"

# Extraer partidos internacionales y goles
try:
    # Encontrar el <li> que contiene 'Partidos internac./goles:'
    stats_li = next((li for li in soup.find_all('li', class_='data-header__label') 
                     if 'Partidos internac./goles:' in li.get_text()), None)
    if stats_li:
        # Encontrar todos los <a> dentro del <li> correspondiente
        stats_links = stats_li.find_all('a', class_='data-header__content--highlight')
        if len(stats_links) == 2:
            partidos = stats_links[0].get_text(strip=True)
            goles = stats_links[1].get_text(strip=True)
        else:
            partidos = "No disponible"
            goles = "No disponible"
    else:
        partidos = goles = "Partidos y goles no encontrados"
except AttributeError:
    partidos = goles = "Partidos y goles no encontrados"

# Extraer el valor del mercado y la última revisión
try:
    # Buscar el <div> que contiene el valor del mercado
    market_value_div = soup.find('div', class_='data-header__box--small')
    if market_value_div:
        # Extraer el valor del mercado
        market_value_span = market_value_div.find('span', class_='waehrung')
        market_value_text = market_value_div.get_text(separator=' ', strip=True)
        
        # Limpiar el valor del mercado de la última revisión
        if market_value_span:
            market_value = market_value_text.split('Última revisión:')[0].strip()
        else:
            market_value = market_value_text.split('Última revisión:')[0].strip()
        
        # Extraer la última revisión
        last_update = market_value_text.split('Última revisión:')[-1].strip() if 'Última revisión:' in market_value_text else "Última revisión no encontrada"
    else:
        market_value = "Valor de mercado no encontrado"
        last_update = "Última revisión no encontrada"
except AttributeError:
    market_value = "Valor de mercado no encontrado"
    last_update = "Última revisión no encontrada"

# Extraer el pie
try:
    pie_li = soup.find('span', string='Pie:')
    if pie_li:
        pie_span = pie_li.find_next_sibling('span', class_='info-table__content--bold')
        pie = pie_span.get_text(strip=True) if pie_span else "Pie no encontrado"
    else:
        pie = "Pie no encontrado"
except AttributeError:
    pie = "Pie no encontrado"

# Extraer el agente
try:
    agente_li = soup.find('span', string='Agente:')
    if agente_li:
        agente_span = agente_li.find_next_sibling('span', class_='info-table__content--bold')
        if agente_span:
            agente = agente_span.get_text(strip=True)
            # Reemplazar "Miembro de su ..." por "Miembro de su familia"
            if "Miembro de su ..." in agente:
                agente = "Miembro de su familia"
        else:
            agente = "Agente no encontrado"
    else:
        agente = "Agente no encontrado"
except AttributeError:
    agente = "Agente no encontrado"

# Extraer el proveedor
try:
    proveedor_li = soup.find('span', string='Proveedor:')
    if proveedor_li:
        proveedor_span = proveedor_li.find_next_sibling('span', class_='info-table__content--bold')
        proveedor = proveedor_span.get_text(strip=True) if proveedor_span else "Proveedor no encontrado"
    else:
        proveedor = "Proveedor no encontrado"
except AttributeError:
    proveedor = "Proveedor no encontrado"

In [68]:
# Imprimir informacion jugador 
print(f"Nombre: {nombre}")
print(f"Equipo: {equipo}")
print(f"Liga: {liga}")
print(f"Fecha de Fichaje: {fecha_fichaje}")
print(f"Contrato hasta: {contrato_hasta}")
print(f"Fecha de Nacimiento: {fecha_nacimiento}")
print(f"Lugar de Nacimiento: {lugar_nacimiento}")
print(f"Nacionalidad: {nacionalidad}")
print(f"Altura: {altura}")
print(f"Posición: {posicion}")
print(f"Selección: {selection}")
print(f"Partidos Internacionales: {partidos}")
print(f"Goles: {goles}")
print(f"Valor de Mercado: {market_value}")
print(f"Última Revisión: {last_update}")
print(f"Pie: {pie}")
print(f"Agente: {agente}")
print(f"Proveedor: {proveedor}")

Nombre: Kylian Mbappé
Equipo: Real Madrid
Liga: LaLiga
Fecha de Fichaje: 01/07/2024
Contrato hasta: 30/06/2029
Fecha de Nacimiento: 20/12/1998 (25)
Lugar de Nacimiento: Paris
Nacionalidad: Francia
Altura: 1,78 m
Posición: Delantero centro
Selección: Francia
Partidos Internacionales: 84
Goles: 48
Valor de Mercado: 180,00 mill. €
Última Revisión: 03/06/2024
Pie: derecho
Agente: Miembro de su familia
Proveedor: Nike


In [70]:
# Guardar en CSV
with open('jugador_info.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Nombre', 'Equipo', 'Liga', 'Fecha de Fichaje', 'Contrato hasta', 'Fecha de Nacimiento', 
                  'Lugar de Nacimiento', 'Nacionalidad', 'Altura', 'Posición', 'Selección', 
                  'Partidos Internacionales', 'Goles', 'Valor de Mercado', 'Última Revisión', 'Pie', 'Agente', 'Proveedor']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerow({
        'Nombre': nombre,
        'Equipo': equipo,
        'Liga': liga,
        'Fecha de Fichaje': fecha_fichaje,
        'Contrato hasta': contrato_hasta,
        'Fecha de Nacimiento': fecha_nacimiento,
        'Lugar de Nacimiento': lugar_nacimiento,
        'Nacionalidad': nacionalidad,
        'Altura': altura,
        'Posición': posicion,
        'Selección': selection,
        'Partidos Internacionales': partidos,
        'Goles': goles,
        'Valor de Mercado': market_value,
        'Última Revisión': last_update,
        'Pie': pie,
        'Agente': agente,
        'Proveedor': proveedor
    })

print("Datos guardados en 'jugador_info.csv'")

Datos guardados en 'jugador_info.csv'


In [71]:
data = pd.read_csv('jugador_info.csv')
data

,Nombre,Equipo,Liga,Fecha de Fichaje,Contrato hasta,Fecha de Nacimiento,Lugar de Nacimiento,Nacionalidad,Altura,Posición,Selección,Partidos Internacionales,Goles,Valor de Mercado,Última Revisión,Pie,Agente,Proveedor
0,Kylian Mbappé,Real Madrid,LaLiga,01/07/2024,30/06/2029,20/12/1998 (25),Paris,Francia,"1,78 m",Delantero centro,Francia,84,48,"180,00 mill. €",03/06/2024,derecho,Miembro de su familia,Nike


In [74]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Nombre                    1 non-null      object
 1   Equipo                    1 non-null      object
 2   Liga                      1 non-null      object
 3   Fecha de Fichaje          1 non-null      object
 4   Contrato hasta            1 non-null      object
 5   Fecha de Nacimiento       1 non-null      object
 6   Lugar de Nacimiento       1 non-null      object
 7   Nacionalidad              1 non-null      object
 8   Altura                    1 non-null      object
 9   Posición                  1 non-null      object
 10  Selección                 1 non-null      object
 11  Partidos Internacionales  1 non-null      int64 
 12  Goles                     1 non-null      int64 
 13  Valor de Mercado          1 non-null      object
 14  Última Revisión           1 no

In [72]:
data.columns

Index(['Nombre', 'Equipo', 'Liga', 'Fecha de Fichaje', 'Contrato hasta',
       'Fecha de Nacimiento', 'Lugar de Nacimiento', 'Nacionalidad', 'Altura',
       'Posición', 'Selección', 'Partidos Internacionales', 'Goles',
       'Valor de Mercado', 'Última Revisión', 'Pie', 'Agente', 'Proveedor'],
      dtype='object')